In [ ]:
import numpy as np
import pandas as pd 
import time
import requests
from fake_useragent import UserAgent

from selenium import webdriver
from selenium.webdriver.chrome.options import Options


ua = UserAgent()
url = "주소"
headers = {'User-Agent' : ua.random}

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")

# 검색하고자 하는 단어 입력

In [ ]:
KEYWORD = '유럽여행'

# url수집
 - 정확도순, 거래글 제외 필터링 시 100페이지이상 넘어가면 동일결과가 나오는 현상 발견

In [ ]:
df = pd.DataFrame(columns=['url'])
PAGE = 1
t1 = time.time()

driver = webdriver.Chrome('chromedriver_91.exe', options=options)

while True:    
    
    # 링크 제시
    base_url = f'https://cafe.naver.com/ca-fe/home/search/articles?q={KEYWORD}&p={PAGE}&em=1'
    driver.get(base_url)
    time.sleep(1)
    
    # 요소 추가
    elements = driver.find_elements_by_xpath('//*[@id="app"]/div/div[2]/div/div[1]/div[3]/div/div[3]/ul/li/div/div/div/a')
    urls = [element.get_attribute('href') for element in elements]
    tmp = pd.DataFrame({'url':urls})
    df = df.append(tmp, ignore_index=True)
    
    # add page & log
    print(f'{PAGE} 페이지 완료!! == 경과시간: {round(time.time() - t1)}초')
    
    # stopping condition (test용)
    if PAGE == 100:
        print('Early Stopping')
        driver.close()
        break
    
    
    # add page
    PAGE += 1

# 세부정보 수집

In [ ]:
df_final = pd.DataFrame(columns = ['제목', '작성자', '일자', '댓글수', '좋아요개수', '조회수', '본문', 'url'])
driver = webdriver.Chrome('chromedriver_91.exe', options=options)
t2 = time.time()

for idx in range(df.shape[0]):
    
    # url 접근
    base_url = df.loc[idx, 'url']
    driver.get(base_url)    
    time.sleep(1)
    
    # url 우회를 위한 trick
    change_tab = driver.window_handles[-1]
    driver.switch_to.window(change_tab)
    driver.switch_to.frame('cafe_main')    
    
    # 정보수집
    headline = driver.find_elements_by_xpath('//*[@id="app"]/div/div/div[2]/div[1]/div[1]/div/h3')[0].text
    writer = driver.find_elements_by_xpath('/html/body/div/div/div/div[2]/div[1]/div[2]/div/div[1]/div/a')[0].text
    date = driver.find_elements_by_xpath('//*[@id="app"]/div/div/div[2]/div[1]/div[2]/div/div[2]/span[1]')[0].text
    content = ' '.join([temp.text for temp in driver.find_elements_by_xpath('/html/body/div/div/div/div[2]/div[2]/div[1]/div/div[1]/div/div/div/div/div/div/p')])
    
    try:
        comment_count = driver.find_elements_by_xpath('/html/body/div/div/div/div[2]/div[1]/div[3]/a[1]/strong')[0].text
    except:
        comment_count = 0
        
    try:
        emotion_count = driver.find_elements_by_xpath('//*[@id="app"]/div/div/div[2]/div[2]/div[3]/div[1]/div/div/a/em[2]')[0].text
    except:
        emotion_count = 0
        
    try:
        view_count = driver.find_elements_by_xpath('/html/body/div/div/div/div[2]/div[1]/div[2]/div/div[2]/span[2]')[0].text.split(' ')[-1]
    except:
        view_count = 0
    
    # df_final에 추가
    tmp = pd.DataFrame({'제목':[headline],
                        '작성자':[writer],
                        '일자':[date],
                        '댓글수':[comment_count],
                        '좋아요개수':[emotion_count],
                        '조회수':[view_count],
                        '본문':[content],
                       'url':[base_url]})
    
    df_final = df_final.append(tmp, ignore_index=True)
    
    # log
    if (idx+1) % 10 == 0:
        print(f'{idx+1}개 완료!! == 경과시간: {round(time.time()-t2)}초')

In [ ]:
df_final